# 訓練データの欠損値を削除する

# テストデータの欠損値を代表値で補完する

In [2]:
import os
os.chdir('../../')

In [3]:
import numpy as np
import pandas as pd
from scr.util import *

In [4]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

In [5]:
# 訓練データの欠損値を削除する
df_train = df_train.dropna(subset=['Age', 'DurationOfPitch', 'NumberOfTrips', 'MonthlyIncome'], axis=0)

In [6]:
df_train.isna().sum()

id                         0
Age                        0
TypeofContact              2
CityTier                   0
DurationOfPitch            0
Occupation                 0
Gender                     0
NumberOfPersonVisiting     0
NumberOfFollowups         29
ProductPitched             0
PreferredPropertyStar      0
NumberOfTrips              0
Passport                   0
PitchSatisfactionScore     0
Designation                0
MonthlyIncome              0
customer_info              0
ProdTaken                  0
dtype: int64

In [7]:
df_test.isna().sum()

id                          0
Age                        93
TypeofContact              12
CityTier                    0
DurationOfPitch           131
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups          24
ProductPitched              0
PreferredPropertyStar       0
NumberOfTrips              42
Passport                    0
PitchSatisfactionScore      0
Designation                 0
MonthlyIncome              49
customer_info               0
dtype: int64

## 正規化 + 欠損値補完 (意味がある特徴量)

In [8]:
df_train['Age'] = df_train['Age'].astype(str).apply(japanese_to_int)
df_test['Age'] = df_test['Age'].astype(str).apply(japanese_to_int)

df_train['TypeofContact'] = df_train['TypeofContact'].fillna('No')
df_test['TypeofContact'] = df_test['TypeofContact'].fillna('No')

df_train['DurationOfPitch'] = df_train['DurationOfPitch'].dropna().apply(convert_to_second)
df_test['DurationOfPitch'] = df_test['DurationOfPitch'].dropna().apply(convert_to_second)

df_train['Gender'] = df_train['Gender'].apply(normalize_gender)
df_test['Gender'] = df_test['Gender'].apply(normalize_gender)

df_train['NumberOfFollowups'] = df_train['NumberOfFollowups'].apply(lambda x: x if x < 100 else x / 100)
df_test['NumberOfFollowups'] = df_test['NumberOfFollowups'].apply(lambda x: x if x < 100 else x / 100)
df_train['NumberOfFollowups'] = df_train['NumberOfFollowups'].fillna(0)
df_test['NumberOfFollowups'] = df_test['NumberOfFollowups'].fillna(0)

df_train['ProductPitched'] = df_train['ProductPitched'].apply(normalize_product_pitched_1)
df_train['ProductPitched'] = df_train['ProductPitched'].apply(normalize_product_pitched_2)
df_test['ProductPitched'] = df_test['ProductPitched'].apply(normalize_product_pitched_1)
df_test['ProductPitched'] = df_test['ProductPitched'].apply(normalize_product_pitched_2)

df_train['NumberOfTrips'] = df_train['NumberOfTrips'].apply(normalize_trips)
df_test['NumberOfTrips'] = df_test['NumberOfTrips'].apply(normalize_trips)

df_train['Designation'] = df_train['Designation'].apply(normalize_designation_1)
df_train['Designation'] = df_train['Designation'].apply(normalize_designation_2)
df_test['Designation'] = df_test['Designation'].apply(normalize_designation_1)
df_test['Designation'] = df_test['Designation'].apply(normalize_designation_2)

df_train['MonthlyIncome'] = df_train['MonthlyIncome'].apply(normalize_monthly_income)
df_test['MonthlyIncome'] = df_test['MonthlyIncome'].apply(normalize_monthly_income)

df_train[['Marry', 'Car', 'Child']] = df_train['customer_info'].apply(divide_customer_info)
df_test[['Marry', 'Car', 'Child']] = df_test['customer_info'].apply(divide_customer_info)
df_train['Marry'] = df_train['Marry'].apply(normalize_info_1)
df_test['Marry'] = df_test['Marry'].apply(normalize_info_1)
df_train['Car'] = df_train['Car'].apply(normalize_info_2)
df_test['Car'] = df_test['Car'].apply(normalize_info_2)
df_train['Child'] = df_train['Child'].apply(normalize_info_3)
df_test['Child'] = df_test['Child'].apply(normalize_info_3)

In [9]:
df_train.isna().sum()

id                        0
Age                       0
TypeofContact             0
CityTier                  0
DurationOfPitch           0
Occupation                0
Gender                    0
NumberOfPersonVisiting    0
NumberOfFollowups         0
ProductPitched            0
PreferredPropertyStar     0
NumberOfTrips             0
Passport                  0
PitchSatisfactionScore    0
Designation               0
MonthlyIncome             0
customer_info             0
ProdTaken                 0
Marry                     0
Car                       0
Child                     0
dtype: int64

In [10]:
df_test.isna().sum()

id                          0
Age                         0
TypeofContact               0
CityTier                    0
DurationOfPitch           131
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups           0
ProductPitched              0
PreferredPropertyStar       0
NumberOfTrips               0
Passport                    0
PitchSatisfactionScore      0
Designation                 0
MonthlyIncome              49
customer_info               0
Marry                       0
Car                         0
Child                       0
dtype: int64

In [11]:
drop_list = ['customer_info']

df_train = df_train.drop(columns=drop_list)
df_test = df_test.drop(columns=drop_list)

## 中央値で補完する

In [12]:
tmp = df_test.loc[df_test['Age'] != 0, 'Age']
df_test.loc[df_test['Age'] == 0, 'Age'] = tmp.median()

In [13]:
tmp = df_test.loc[~(df_test['DurationOfPitch'].isna()), 'DurationOfPitch']
df_test.loc[df_test['DurationOfPitch'].isna(), 'DurationOfPitch'] = tmp.median()

In [14]:
# tmp = df_test.loc[~(df_test['NumberOfFollowups'].isna()), 'NumberOfFollowups']
# df_test.loc[df_test['NumberOfFollowups'].isna(), 'NumberOfFollowups'] = tmp.median()

In [15]:
tmp = df_test.loc[~(df_test['NumberOfTrips'] == 'nan'), 'NumberOfTrips']
df_test.loc[df_test['NumberOfTrips'] == 'nan', 'NumberOfTrips'] = tmp.median()

In [16]:
tmp = df_test.loc[~(df_test['MonthlyIncome'].isna()), 'MonthlyIncome']
df_test.loc[df_test['MonthlyIncome'].isna(), 'MonthlyIncome'] = tmp.median()

In [17]:
print(len(df_train))
print(len(df_test))

3237
3489


In [18]:
df_train.isna().sum()

id                        0
Age                       0
TypeofContact             0
CityTier                  0
DurationOfPitch           0
Occupation                0
Gender                    0
NumberOfPersonVisiting    0
NumberOfFollowups         0
ProductPitched            0
PreferredPropertyStar     0
NumberOfTrips             0
Passport                  0
PitchSatisfactionScore    0
Designation               0
MonthlyIncome             0
ProdTaken                 0
Marry                     0
Car                       0
Child                     0
dtype: int64

In [19]:
df_test.isna().sum()

id                        0
Age                       0
TypeofContact             0
CityTier                  0
DurationOfPitch           0
Occupation                0
Gender                    0
NumberOfPersonVisiting    0
NumberOfFollowups         0
ProductPitched            0
PreferredPropertyStar     0
NumberOfTrips             0
Passport                  0
PitchSatisfactionScore    0
Designation               0
MonthlyIncome             0
Marry                     0
Car                       0
Child                     0
dtype: int64

# 数値変換 (DurationOfPitch, MonthlyIncome)

In [20]:
df_train['DurationOfPitch'] = np.log1p(df_train['DurationOfPitch'])
df_train['MonthlyIncome'] = np.log1p(df_train['MonthlyIncome'])

df_test['DurationOfPitch'] = np.log1p(df_test['DurationOfPitch'])
df_test['MonthlyIncome'] = np.log1p(df_test['MonthlyIncome'])

## 年代特徴量作成

In [21]:
def age_to_agegroup(age):
    if age == 0:
        return np.nan    
    elif age < 20:
        return "10s"
    elif age < 30:
        return "20s"
    elif age < 40:
        return "30s"
    elif age < 50:
        return "40s"
    elif age < 60:
        return "50s"
    else:
        return "60s"

df_train['AgeGroup'] = df_train['Age'].apply(age_to_agegroup)
df_test['AgeGroup'] = df_test['Age'].apply(age_to_agegroup)

## 確認

In [22]:
display(df_train.head())
display(df_test.head())

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,...,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,Marry,Car,Child,AgeGroup
0,0,50,Self Enquiry,2,6.803505,Large Business,male,1.0,4.0,Basic,...,5,1,4,Executive,12.444719,1,Single,No Car,0_child,50s
1,1,56,Company Invited,1,6.734592,Salaried,male,1.0,4.0,Standard,...,2,1,4,Senior Manager,12.910348,0,Divorced,Has Car,0_child,50s
3,3,37,Self Enquiry,2,6.985642,Small Business,female,1.0,3.0,Standard,...,1,0,5,Senior Manager,12.697122,0,Divorced,Has Car,0_child,30s
4,4,48,Company Invited,3,6.928538,Small Business,female,1.0,3.0,Basic,...,4,0,4,Executive,12.462403,1,Single,Has Car,0_child,40s
5,5,19,Self Enquiry,2,6.867974,Small Business,male,1.0,3.0,Basic,...,2,0,4,Executive,12.468441,0,Single,No Car,0_child,10s


,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,Marry,Car,Child,AgeGroup
0,3489,48,Self Enquiry,2,6.660575,Small Business,male,1.0,4.0,Super Deluxe,3.0,7,0,3,AVP,13.116247,Married,Has Car,0_child,40s
1,3490,30,Self Enquiry,2,6.580639,Small Business,female,1.0,4.0,Standard,3.0,4,1,3,Senior Manager,12.611541,Married,No Car,0_child,30s
2,3491,25,Self Enquiry,1,6.293419,Salaried,female,1.0,4.0,Basic,3.0,1,0,3,Executive,12.468441,Divorced,No Car,0_child,20s
3,3492,21,Company Invited,2,6.042633,Salaried,male,1.0,4.0,Basic,4.0,1,0,3,Senior Manager,12.467960,Divorced,Has Car,0_child,20s
4,3493,41,Company Invited,1,6.042633,Salaried,male,1.0,4.0,Basic,3.0,1,0,4,Executive,12.501838,Single,Has Car,0_child,40s


# CSV出力

In [23]:
df_train.to_csv('data/feature_engineered/null_del_xxx/train_null_del.csv', index=False)
df_test.to_csv('data/feature_engineered/null_del_xxx/test_null_median.csv', index=False)